# Food Classification with TensorFlow Hub - TFLite

## Setup

In [ ]:
from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices()) 
# my output was => ['/device:CPU:0']
# good output must be => ['/device:CPU:0', '/device:GPU:0']

In [ ]:
try:
    %tensorflow_version 2.x
except:
    pass

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
# from google.colab import files
import tensorflow as tf
# import tensorflow_hub as hub
import random
import shutil
from shutil import copyfile
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from keras.applications.mobilenet_v2 import MobileNetV2
# from efficientnet.tfkeras import EfficientNetB3
from sklearn.metrics import classification_report, confusion_matrix
import itertools
from PIL import Image
import scipy

print('\u2022 Using TensorFlow Version:', tf.__version__)
# print('\u2022 Using TensorFlow Hub Version: ', hub.__version__)
print('\u2022 GPU Device Found.' if tf.config.list_physical_devices('GPU') else '\u2022 GPU Device Not Found. Running on CPU')

### Format the Data


In [ ]:
# from google.colab import drive
# import zipfile
# drive.mount('/content/drive')

In [ ]:
cwd = os.getcwd()
print(cwd)

In [ ]:
# zip_ref = zipfile.ZipFile('/content/drive/Shareddrives/ML Capstone Project/food_datasets.zip', 'r')
# zip_ref.extractall(f'{cwd}/')
# zip_ref.close()

# ! pip install kaggle
# ! mkdir ~/.kaggle
# ! cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle datasets download -d utkarshsaxenadn/fast-food-classification-dataset
# ! unzip fast-food-classification-dataset.zip

In [ ]:
TRAINING_DIR = f'../Dataset/food_datasets/train'
VALIDATION_DIR = f'../Dataset/food_datasets/valid'
TEST_DIR = f'../Dataset/food_datasets/test'

In [ ]:
def count_images_in_directory(directory):
  folders = os.listdir(directory)
  folders.sort()
  for folder in folders:
    x = os.listdir(os.path.join(directory, folder))
    print(f'{folder}: {len(x)} images')

In [ ]:
print('TRAINING:')
count_images_in_directory(TRAINING_DIR)

print('\nVALIDATION:')
count_images_in_directory(VALIDATION_DIR)

print('\nTEST:')
count_images_in_directory(TEST_DIR)

In [ ]:
folders = os.listdir(TRAINING_DIR)
folders.sort()
for folder in folders:
  items = os.path.join(TRAINING_DIR, folder)
  item = os.listdir(items)
  index = random.randint(0, len(item) - 1)   # get a random index
  print(f'\nSample {folder} image:')
  print(f'{item[index]}')
  plt.figure(figsize=(3, 3))
  plt.imshow(load_img(f'{os.path.join(items, os.listdir(items)[index])}'))
  plt.show()

In [ ]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
  train_datagen = ImageDataGenerator(rescale = 1/255,
                                     rotation_range = 40,
                                     width_shift_range = 0.2,
                                     height_shift_range = 0.2,
                                     shear_range = 0.2,
                                     zoom_range = 0.2,
                                     horizontal_flip = True)
  train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=32,
                                                      class_mode='categorical',
                                                      target_size=(224, 224))
  
  validation_datagen    = ImageDataGenerator(rescale = 1/255)
  validation_generator  = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                batch_size=32,
                                                                class_mode='categorical',
                                                                target_size=(224,224))

  return train_generator, validation_generator

In [ ]:
train_generator, validation_generator = train_val_generators(TRAINING_DIR, VALIDATION_DIR)

## Defining the Transfer Learning Model

In [ ]:
def create_pretrained_model():
  pretrained_model = MobileNetV2(include_top=False,
                                   weights="imagenet",
                                   input_shape=(224,224,3),
                                   classes=30)
                                   
  for layer in pretrained_model.layers:
    layer.trainable = False
  return pretrained_model

In [ ]:
pretrained_model = create_pretrained_model()

In [ ]:
pretrained_model.summary()

In [ ]:
def output_of_last_layer(pretrained_model):
  last_desired_layer = pretrained_model.get_layer('out_relu') #avg_pool diganti top_activation biar bisa ditambah layer CNN sama MaxPool
  print('Last layer output shape: ', last_desired_layer.output_shape)
  last_output = last_desired_layer.output
  print('Last layer output: ', last_output)
  return last_output

In [ ]:
last_output = output_of_last_layer(pretrained_model)

## Compiling the Model

In [ ]:
def create_model(pretrained_model, last_output):
    x = layers.Conv2D(512, (2,2), activation = 'relu')(last_output)
    x = layers.Conv2D(512, (2,2), activation = 'relu')(x)
    x = layers.Conv2D(256, (2,2), activation = 'relu')(x)
    x = layers.Conv2D(256, (2,2), activation = 'relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(256, activation = 'relu')(x)
    x = layers.Dense(256, activation = 'relu')(x)
    x = layers.Dense(128, activation = 'relu')(x)
    x = layers.Dense(train_generator.num_classes, activation = 'softmax')(x)

    model = Model(inputs=pretrained_model.input, outputs=x)

    from tensorflow.keras.optimizers import Adam
    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy']) 
    return model

In [ ]:
model = create_model(pretrained_model, last_output)

model.summary()

## Training the Model

In [ ]:
history = model.fit(train_generator,
                    epochs=20,
                    verbose=1,
                    validation_data=validation_generator)

In [ ]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs=range(len(acc)) 

# Plot training and validation accuracy per epoch
plt.plot(epochs, acc, 'r', 'Training Accuracy')
plt.plot(epochs, val_acc, 'b', 'Validation Accuracy')
plt.title('Training and validation accuracy')
plt.show()
print('')

# Plot training and validation loss per epoch
plt.plot(epochs, loss, 'r', 'Training Loss')
plt.plot(epochs, val_loss, 'b', 'Validation Loss')
plt.show()

In [ ]:
# calculating the recall, precision and f1-score
test_datagen    = ImageDataGenerator(rescale = 1/255)
test_generator = test_datagen.flow_from_directory(
        TEST_DIR,  # This is the source directory for training images
        target_size=(224, 224),  # All images will be resized to 150x150
        batch_size=16,
        shuffle=False,
        class_mode='categorical')

test_score = model.evaluate(test_generator)
print('[INFO] accuracy: {:.2f}%'.format(test_score[1] * 100))
print('[INFO] Loss: ',test_score[0])

In [ ]:

# Plot the confusion matrix. Set Normalize = True/False
def plot_confusion_matrix(cm, classes, normalize=True, title='Confusion matrix', cmap=plt.cm.Blues):
    '''
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    '''
    plt.figure(figsize=(15, 15))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm = np.around(cm, decimals=2)
        cm[np.isnan(cm)] = 0.0
        print('Normalized confusion matrix')
    else:
        print('Confusion matrix, without normalization')
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j], horizontalalignment='center', color='white' if cm[i, j] > thresh else 'black')
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Print the Target names
target_names = []
for key in test_generator.class_indices:
    target_names.append(key)

# Confusion Matrix
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
cm = confusion_matrix(test_generator.classes, y_pred)
print(confusion_matrix(test_generator.classes, y_pred))
plot_confusion_matrix(cm, target_names, title='Confusion Matrix')

# Print Classification Report
print('Classification Report')
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

## Export the Model

In [ ]:
FC_SAVED_MODEL = 'fc_saved_model'

Export the SavedModel

In [ ]:
model.save(FC_SAVED_MODEL)

In [ ]:
# %%bash -s $FC_SAVED_MODEL
# saved_model_cli show --dir $1 --tag_set serve --signature_def serving_default

In [ ]:
loaded = tf.saved_model.load(FC_SAVED_MODEL)

In [ ]:
print(list(loaded.signatures.keys()))
infer = loaded.signatures['serving_default']
print(infer.structured_input_signature)
print(infer.structured_outputs)

## Convert Using TFLite's Converter

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(FC_SAVED_MODEL)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

In [ ]:
tflite_model_file = 'fc_model.tflite'

with open(tflite_model_file, 'wb') as f:
    f.write(tflite_model)

Create a file to save the labels.

In [ ]:
class_names = folders #disesuaikan
print(class_names)

In [ ]:
with open('labels.txt', 'w') as f:
    f.write('\n'.join(class_names))